In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
from collections import Counter
import math

# Init Variables

In [ ]:
NODE_NUMBER = 1000
MAX_EDGE_NUMBER = (NODE_NUMBER) * (NODE_NUMBER - 1) / 2
EDGE_PROBABILITY = 0.05
EDGE_NUMBER = int(MAX_EDGE_NUMBER * EDGE_PROBABILITY)  # Approximately 0.05 of max possible edge number
# GRAPH_NUMBER = 100
GRAPH_NUMBER = 10
# SIMULATION_NUMBER = 100
SIMULATION_NUMBER = 1
COLOR = ['red', 'green', 'blue', 'gray', 'yellow', 'brown', 'black']

# Functions

### Eigvals

###### Adjacency

In [ ]:
def get_adjacency_eigvals(graph):
    L = nx.adjacency_matrix(graph)
    e = np.linalg.eigvals(L.toarray())
    e = list(sorted(e))
    return e

###### Laplacian

In [ ]:
def get_laplacian_evigal(graph): # Laplacian matrix
    L = nx.laplacian_matrix(graph)
    e = np.linalg.eigvals(L.toarray())
    e = list(sorted(e))
    return e

### Spectral Gap

In [ ]:
def get_spectral_gap(graph):
    eigvals = get_adjacency_eigvals(graph)
    max_index = len(eigvals) - 1
    maximum = eigvals[max_index]
    second_max = eigvals[max_index - 1]
    diff = maximum - second_max
    return diff

### Algebraic Connectivity

In [ ]:
def get_algebraic_connectivity(graph):
    return nx.algebraic_connectivity(graph)

### check for connection with Algebraic Connectivity

In [ ]:
def is_graph_connected(graph):
    return get_algebraic_connectivity(graph) > 0

### Trace Power S

In [ ]:
def get_trace_power_s(graph, power=2): # number of walks with len s (power) in graph
    eigvals = get_adjacency_eigvals(graph)
    eigvals_powers = np.power(eigvals, power)
    summation = np.sum(eigvals_powers)
    return summation

### Phi s (Average Trace Power S)

In [ ]:
def get_phi_s(graph): # average number of walks with len s (power) in graph
    eigvals = get_adjacency_eigvals(graph)
    eigvals_powers = np.power(eigvals, power)
    summation = np.sum(eigvals_powers)
    avg = summation / len(eigvals)
    return summation

### Centrality of Global Subgraph

In [ ]:
def get_centrality_of_global_subgraph(graph):
    eigvals = get_adjacency_eigvals(graph)
    summation = sum([math.exp(value) for value in eigvals])
    return summation

### Average Eigvals

In [ ]:
def get_average_eigvals(graph):
    summation = get_centrality_of_global_subgraph(graph)
    ln = math.log(summation)
    return ln

## Automorphism

In [ ]:
def get_all_automorphism(graph): # automorphism is isomorphism for a graph with itself
    dictionary = nx.vf2pp_all_isomorphisms(graph, graph)
    return list(dictionary) # return a list of mapping (return a list of dictionary)

### Node Similarity (Vertex Transitivity)

In [ ]:
def is_node_similar(graph):
    automorphisms = get_all_automorphism(graph)
    for u,v in graph.edges:
        if not any(auto[u] == v for auto in automorphisms):
            return False
    return True

### Symmetry (Edge Transitivity)

In [ ]:
def is_symmetry(graph):
    automorphisms = get_all_automorphism(graph)
    for u, v in graph.edges:
        for x, y in graph.edges:
            if not any(auto[u] == x and auto[v] ==y for auto in automorphisms):
                return False
    return True

### Laplacian Energy

In [ ]:
def get_laplacian_energy(graph):
    if not nx.is_connected(graph):
        raise Exception("Graph Must Be Connected")
    if nx.is_directed(graph):
        raise Exception("Graph Must Be Undirected")
    eigenvalues = get_adjacency_eigvals(graph)
    laplacian_energy = sum([abs(value) for value in eigenvalues])
    return laplacian_energy

In [ ]:
def get_laplacian_energy2(graph):
    if not nx.is_connected(graph):
        raise Exception("Graph Must Be Connected")
    if nx.is_directed(graph):
        raise Exception("Graph Must Be Undirected")
    eigenvalues = get_adjacency_eigvals(graph)
    summation = sum([value if value > 0 else 0 for value in eigenvalues])
    laplacian_energy = summation * 2
    return laplacian_energy

In [ ]:
def get_laplacian_energy3(graph):
    if not nx.is_connected(graph):
        raise Exception("Graph Must Be Connected")
    if nx.is_directed(graph):
        raise Exception("Graph Must Be Undirected")
    eigenvalues = get_laplacian_evigal(graph)
    n = graph.number_of_nodes()
    m = graph.number_of_edges()
    constant = (2 * m) / n
    laplacian_energy = sum([abs(value - constant) for value in eigenvalues])
    return laplacian_energy

### Plotting Graphs

In [ ]:
def show_graph(graph, path=None):
    pos = nx.circular_layout(graph)
    plt.figure(figsize = (12, 12))
    nx.draw_networkx(graph, pos)
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

### Degree Distribution

In [ ]:
def degree_distribution(graph, path=None, style='-o'):
    degrees = [graph.degree(n) for n in graph.nodes()]
    degrees = list(sorted(degrees))
    degree_freq_dic = Counter(degrees)
    x_axis = degree_freq_dic.keys()
    y_axis = degree_freq_dic.values()
    y_axis = np.array(list(y_axis)) / len(degrees)
    
    plt.title('Degree Distribution')
    plt.xlabel("Degree")
    plt.ylabel("Frequesncy")
    plt.plot(x_axis, y_axis, style, label='degree probability')
    
    upper_y = np.array([0, max(y_axis)])
    avg = np.average(degrees)
    upper_x = np.array([avg, avg])
    plt.plot(upper_x, upper_y, color='red', linestyle='-.', label='mean')
    plt.legend(loc='best') # setting best location for labels
    
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

### Double-Log

In [ ]:
def double_log(graph, path=None, style='-o'):
    degrees = [graph.degree(n) for n in graph.nodes()]
    degrees = list(sorted(degrees))
    degree_freq_dic = Counter(degrees)
    unique_degrees = list(degree_freq_dic.keys())
    frequency = list(degree_freq_dic.values())
    x_axis = np.log(unique_degrees)
    y_axis = np.log(frequency)
    y_axis = np.array(list(y_axis)) / len(degrees)
    plt.xlabel("Degree")
    plt.ylabel("Degree Distribution")
    plt.title('Double Log')
    plt.plot(x_axis, y_axis, style, label='degree distribution')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

## Comparing Plots

In [ ]:
def compare_datas(datas, labels, x_label='', y_label='', title='', style='-o', color=COLOR, path=None):
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    for i in range(len(datas)):
        x_axis = list(range(len(datas[i])))
        plt.plot(x_axis, datas[i], style, label=labels[i], color=color[i])
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

## Data Details

### c.i plots

In [ ]:
def coefficient_interval_plot(data, path=None, alpha=0.95):
    x = np.array([i for i in range(len(data))])
    y = np.array(data)
    # plotting
    plt.plot(y, x,'o', color='blue', label='data')
    
    # confidence intervals
    p = ((1.0-alpha)/2.0) * 100
    # percentile function returns the numbers which that percent of 
    # the array elements areless equal then that number
    lower =  np.percentile(y, p) 
    p = (alpha+((1.0-alpha)/2.0)) * 100
    upper =  np.percentile(y, p)
#     print(f"\n{alpha*100} confidence interval {lower} and {upper}")
    
    # c.i upper & lower
    upper_y = np.array([0, len(data)])
    upper_x = np.array([upper, upper])
    plt.plot(upper_x, upper_y, color='red', linestyle='-.', label='upper c.i')
    
    lower_y = np.array([0, len(data)])
    lower_x = np.array([lower, lower])
    plt.plot(lower_x, lower_y, color='orange', linestyle='-.', label='lower c.i')
    
    ci_x = np.array([lower, upper])
    ci_y = np.array([0, 0])
    plt.plot(ci_x, ci_y, '-', color='green', label='c.i')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def coefficient_interval_plot2(data, path=None, alpha=0.95):
    x = np.array(list(range(len(data))))
    y = np.array(data)
    # Plotting data
    plt.plot(x, y, '-o', color='red', label='data')
    
    # confidence intervals
    ci = (1.0-alpha) * np.std(y) / np.mean(y)
    mean = np.mean(y)
    avg = [mean for i in range(len(data))]
    
    # Plot the confidence interval
    plt.fill_between(x, (avg-ci), (avg+ci), color='blue', alpha=0.1)
    plt.plot(x, (avg-ci), '--', color='blue', label='-*ci')
    plt.plot(x, (avg+ci), '--', color='blue', label='+*ci')
    plt.fill_between(x, (avg-2*ci), (avg+2*ci), color='green', alpha=.1)
    plt.plot(x, (avg-2*ci), '--', color='green', label='-2*ci')
    plt.plot(x, (avg+2*ci), '--', color='green', label='+2*ci')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def coefficient_interval_plot3(data, path=None, alpha=0.95):
    x = np.array(list(range(len(data))))
    y = np.array(data)
    # Plotting data
    plt.plot(x, y, '-o', color='red', label='data')
    
    # Define the confidence interval
    ci = (1.0-alpha) * np.std(y) / np.mean(y)
    
    # Plot the confidence interval
    plt.fill_between(x, (y-ci), (y+ci), color='blue', alpha=0.1)
    plt.plot(x, (y-2*ci), '--', color='blue', label='-*ci')
    plt.plot(x, (y+2*ci), '--', color='blue', label='+*ci')
    plt.fill_between(x, (y-2*ci), (y+2*ci), color='green', alpha=.1)
    plt.plot(x, (y-2*ci), '--', color='green', label='-2*ci')
    plt.plot(x, (y+2*ci), '--', color='green', label='+2*ci')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

### Mean & Variance

In [ ]:
def get_details(data):
    print('mean: ', np.mean(data))
    print('variance: ', np.var(data))

# Making Graphs

### Small-World (watts-storgatz)

In [ ]:
sw_seed_values = random.sample(range(1, 100000), GRAPH_NUMBER) # generating GRAPH_NUMBER unique random number to be used as seed

small_worlds = []

k = round(((2 * EDGE_NUMBER) / NODE_NUMBER))

for i in range(GRAPH_NUMBER):
    print('Graph No: ', i)
    # we want to have EDGE_NUMBER edge, and in the base graph we have k degree
    # for each node. And we know summation of node degrees, is 2 * EDGE_NUMBER
    # so we have tohave k = (2 * EDGE_NUMBER) / NODE_NUMBER  for each node.
    rewiring_probability = random.uniform(0.2, 0.3)
    graph = graph = nx.watts_strogatz_graph(n=NODE_NUMBER, k=k, p=rewiring_probability, seed=sw_seed_values[i])
    small_worlds.append(graph)
#     show_graph(graph)

#### Degree Distribution

In [ ]:
for graph in small_worlds:
    degree_distribution(graph)

### Scale-Free (barabasi-albert)

In [ ]:
sf_seed_values = random.sample(range(1, 100000), GRAPH_NUMBER) # generating GRAPH_NUMBER unique random number to be used as seed

m = round(((EDGE_NUMBER) / NODE_NUMBER))

scale_frees = []

for i in range(GRAPH_NUMBER):
    print('Graph no: ', i)
    graph = nx.barabasi_albert_graph(n=NODE_NUMBER, m=m, seed=sf_seed_values[i], initial_graph=None)
    scale_frees.append(graph)
    
#     show_graph(graph)

#### Degree Distribution

In [ ]:
for graph in scale_frees:
    degree_distribution(graph, style='o')

#### Log-Log Plot

In [ ]:
for graph in scale_frees:
    double_log(graph, style='o')

### Random

In [ ]:
er_seed_values = random.sample(range(1, 100000), GRAPH_NUMBER) # generating GRAPH_NUMBER unique random number to be used as seed

randoms = []
for i in range(GRAPH_NUMBER):
    print('graph no: ', i)
    graph = nx.erdos_renyi_graph(NODE_NUMBER, EDGE_PROBABILITY, seed=er_seed_values[i])
    randoms.append(graph)

### Degree Distribution

In [ ]:
for graph in randoms:
    degree_distribution(graph)

# Algebric Connectivity

### Calculating

In [ ]:
# ac = algebraic connectivity
small_worlds_ac = []
scale_frees_ac = [] 
randoms_ac = []

for i in range(GRAPH_NUMBER):
    small_worlds_ac.append(get_algebraic_connectivity(small_worlds[i]))
    scale_frees_ac.append(get_algebraic_connectivity(scale_frees[i]))
    randoms_ac.append(get_algebraic_connectivity(randoms[i]))

### small world

In [ ]:
get_details(small_worlds_ac)

In [ ]:
coefficient_interval_plot(small_worlds_ac)

In [ ]:
coefficient_interval_plot2(small_worlds_ac)

In [ ]:
coefficient_interval_plot3(small_worlds_ac)

### Scale Free

In [ ]:
get_details(scale_frees_ac)

In [ ]:
coefficient_interval_plot(scale_frees_ac)

In [ ]:
coefficient_interval_plot2(scale_frees_ac)

In [ ]:
coefficient_interval_plot3(scale_frees_ac)

### Random

In [ ]:
get_details(randoms_ac)

In [ ]:
coefficient_interval_plot(randoms_ac)

In [ ]:
coefficient_interval_plot2(randoms_ac)

In [ ]:
coefficient_interval_plot3(randoms_ac)

# Comparing

In [ ]:
datas = [small_worlds_ac, scale_frees_ac, randoms_ac]
lables = ['small world', 'scale free', 'random']
compare_datas(datas, lables, x_label='Graph Number'
              , y_label='Algebraic Connectivity', title='Comparing algebraic connectivity of graphs')

# Spectral Gap

### Calculating

In [ ]:
# sg = spectral gap
sg_small_world = []
sg_scale_free = []
sg_random = []

for i in range(GRAPH_NUMBER):
    sg_small_world.append(get_spectral_gap(small_worlds[i]))
    sg_scale_free.append(get_spectral_gap(scale_frees[i]))
    sg_random.append(get_spectral_gap(randoms[i]))

### Small World

In [ ]:
get_details(sg_small_world)

In [ ]:
coefficient_interval_plot(sg_small_world)

In [ ]:
coefficient_interval_plot2(sg_small_world)

In [ ]:
coefficient_interval_plot3(sg_small_world)

### Scale Free

In [ ]:
get_details(sg_scale_free)

In [ ]:
coefficient_interval_plot(sg_scale_free)

In [ ]:
coefficient_interval_plot2(sg_scale_free)

In [ ]:
coefficient_interval_plot3(sg_scale_free)

### Random

In [ ]:
get_details(sg_random)

In [ ]:
coefficient_interval_plot(sg_random)

In [ ]:
coefficient_interval_plot2(sg_random)

In [ ]:
coefficient_interval_plot3(sg_random)

### Comparing

In [ ]:
datas = [sg_small_world, sg_scale_free, sg_random]
lables = ['small world', 'scale free', 'random']
compare_datas(datas, lables, x_label='Graph Number'
              , y_label='Spectral Gap', title='Comparing Spectral Gap of graphs')

# Centraliti Of Global Subgraph

### Calculate

In [ ]:
# cgs = centrality of global subgraph
cgs_small_world = []
cgs_scale_free = []
cgs_random = []
for i in range(GRAPH_NUMBER):
    cgs_small_world.append(get_centrality_of_global_subgraph(small_worlds[i]))
    cgs_scale_free.append(get_centrality_of_global_subgraph(scale_frees[i]))
    cgs_random.append(get_centrality_of_global_subgraph(randoms[i]))

### Small World

In [ ]:
get_details(cgs_small_world)

In [ ]:
coefficient_interval_plot(cgs_small_world)

In [ ]:
coefficient_interval_plot2(cgs_small_world)

In [ ]:
coefficient_interval_plot3(cgs_small_world)

### Compare

In [ ]:
datas = [cgs_small_world, cgs_scale_free, cgs_random]
lables = ['small-world', 'scale-free', 'random']
compare_datas(datas, lables, x_label='Graph Number'
              , y_label='Centrality Of Global Subgraph', title='Comparing Centrality Of Global Subgraph of graphs')

# Average Eigvals

### Calculating

In [ ]:
# ae = average eigvals
ae_small_world = []
ae_scale_free = []
ae_random = []
for i in range(GRAPH_NUMBER):
    ae_small_world.append(get_average_eigvals(small_worlds[i]))
    ae_scale_free.append(get_average_eigvals(scale_frees[i]))
    ae_random.append(get_average_eigvals(randoms[i]))

### small world

In [ ]:
get_details(ae_small_world)

In [ ]:
coefficient_interval_plot(ae_small_world)

In [ ]:
coefficient_interval_plot2(ae_small_world)

In [ ]:
coefficient_interval_plot3(ae_small_world)

### scale free

In [ ]:
get_details(ae_scale_free)

In [ ]:
coefficient_interval_plot(ae_scale_free)

In [ ]:
coefficient_interval_plot2(ae_scale_free)

In [ ]:
coefficient_interval_plot3(ae_scale_free)

### random

In [ ]:
get_details(ae_random)

In [ ]:
coefficient_interval_plot(ae_random)

In [ ]:
coefficient_interval_plot2(ae_random)

In [ ]:
coefficient_interval_plot3(ae_random)

### Compare

In [ ]:
datas = [ae_small_world, ae_scale_free, ae_random]
lables = ['small-world', 'scale-free', 'random']
compare_datas(datas, lables, x_label='Graph Number'
              , y_label='Average eigvals', title='Comparing Average eigvals of graphs')

# Node Similarity

### Calculating

In [ ]:
ns_small_world = []
ns_scale_free = []
ns_random = []
for i in range(GRAPH_NUMBER):
    ns_small_world.append(is_node_similar(small_worlds[i]))
    ns_scale_free.append(is_node_similar(scale_frees[i]))
    ns_random.append(is_node_similar(randoms[i]))

### small world

In [ ]:
total_number = GRAPH_NUMBER
node_similar_number = sum(ns_small_world)
ns_probability = (node_similar_number / total_number) * 100
print(f'node similar: {ns_probability}%')

### scale free

In [ ]:
total_number = GRAPH_NUMBER
node_similar_number = sum(ns_scale_free)
ns_probability = (node_similar_number / total_number) * 100
print(f'node similar: {ns_probability}%')

### random

In [ ]:
total_number = GRAPH_NUMBER
node_similar_number = sum(ns_random)
ns_probability = (node_similar_number / total_number) * 100
print(f'node similar: {ns_probability}%')

# Symmetry

### Calculating

In [ ]:
sym_small_world = []
sym_scale_free = []
sym_random = []
for i in range(GRAPH_NUMBER):
    sym_small_world.append(is_symmetry(small_worlds[i]))
    sym_scale_free.append(is_symmetry(scale_frees[i]))
    sym_random.append(is_symmetry(randoms[i]))

### small world

In [ ]:
total_number = GRAPH_NUMBER
symmetry_number = sum(sym_small_world)
sym_probability = (symmetry_number / total_number) * 100
print(f'symmetry: {sym_probability}%')

### scale free

In [ ]:
total_number = GRAPH_NUMBER
symmetry_number = sum(sym_scale_free)
sym_probability = (symmetry_number / total_number) * 100
print(f'symmetry: {sym_probability}%')

### random

In [ ]:
total_number = GRAPH_NUMBER
symmetry_number = sum(sym_random)
sym_probability = (symmetry_number / total_number) * 100
print(f'symmetry: {sym_probability}%')

# Laplacian Energy

### Calculating

In [ ]:
# le = laplacian energy
le_small_world = []
le_scale_free = []
le_random = []
for i in range(GRAPH_NUMBER):
    le_small_world.append(get_laplacian_energy(small_worlds[i]))
    le_scale_free.append(get_laplacian_energy(scale_frees[i]))
    le_random.append(get_laplacian_energy(randoms[i]))

### small world

In [ ]:
get_details(le_small_world)

In [ ]:
coefficient_interval_plot(le_small_world)

In [ ]:
coefficient_interval_plot2(le_small_world)

In [ ]:
coefficient_interval_plot3(le_small_world)

### scale free

In [ ]:
get_details(le_scale_free)

In [ ]:
coefficient_interval_plot(le_scale_free)

In [ ]:
coefficient_interval_plot2(le_scale_free)

In [ ]:
coefficient_interval_plot3(le_scale_free)

### random

In [ ]:
get_details(le_random)

In [ ]:
coefficient_interval_plot(le_random)

In [ ]:
coefficient_interval_plot2(le_random)

In [ ]:
coefficient_interval_plot3(le_random)

### compare

In [ ]:
datas = [le_small_world, le_scale_free, le_random]
lables = ['small-world', 'scale-free', 'random']
compare_datas(datas, lables, x_label='Graph Number'
              , y_label='Laplacian Energy', title='Comparing Laplacian Energy of graphs')